In [7]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
%matplotlib inline

In [8]:
def readTrain():
  train = pd.read_csv("S&P 500.csv")
  return train

In [9]:
def augFeatures(train):
  train["Date"] = pd.to_datetime(train["Date"])
  train["year"] = train["Date"].dt.year
  train["month"] = train["Date"].dt.month
  train["date"] = train["Date"].dt.day
  train["day"] = train["Date"].dt.dayofweek
  return train

In [10]:
def normalize(train):
  train = train.drop(["Date"], axis=1)
  train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
  return train_norm

In [11]:
def buildTrain(train, pastDay=30, futureDay=5):
  X_train, Y_train = [], []
  for i in range(train.shape[0]-futureDay-pastDay):
    X_train.append(np.array(train.iloc[i:i+pastDay]))
    Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay]["Adj Close"]))
  return np.array(X_train), np.array(Y_train)

In [12]:
def shuffle(X,Y):
  np.random.seed(10)
  randomList = np.arange(X.shape[0])
  np.random.shuffle(randomList)
  return X[randomList], Y[randomList]

In [13]:
def splitData(X,Y,rate):
  X_train = X[int(X.shape[0]*rate):]
  Y_train = Y[int(Y.shape[0]*rate):]
  X_val = X[:int(X.shape[0]*rate)]
  Y_val = Y[:int(Y.shape[0]*rate)]
  return X_train, Y_train, X_val, Y_val

In [14]:
# read SPY.csv
train = readTrain()

# Augment the features (year, month, date, day)
train_Aug = augFeatures(train)

# Normalization
train_norm = normalize(train_Aug)

# build Data, use last 30 days to predict next 5 days
X_train, Y_train = buildTrain(train_norm, 30, 5)

# shuffle the data, and random seed is 10
X_train, Y_train = shuffle(X_train, Y_train)

# split training data and validation data
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)


In [15]:
def buildOneToOneModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2],return_sequences=True))
  # output shape: (1, 1)
  model.add(TimeDistributed(Dense(1)))    # or use model.add(Dense(1))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [16]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 1, 1)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,np.newaxis]
Y_val = Y_val[:,np.newaxis]

model = buildOneToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=True, input_shape=(1, 10))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 1, 10)             840       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 1, 1)              11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5710 samples, validate on 634 samples
Epoch 1/1000
5710/5710 [==============================] - 1s 208us/step - loss: 0.0720 - val_loss: 0.0532
Epoch 2/1000
5710/5710 [==============================] - 0s 13us/step - loss: 0.0424 - val_loss: 0.0289
Epoch 3/1000
5710/5710 [==============================] - 0s 7us/step - loss: 0.0203 - val_loss: 0.0114
Epoch 4/1000
5710/5710 [==============================] - 0s 13us/step - loss: 0.0065 - val_loss: 0.0027
Epoch 5/1000
5710/5710 [==============================] - 0s 11us/step - 

5710/5710 [==============================] - 0s 12us/step - loss: 3.7505e-05 - val_loss: 3.9341e-05
Epoch 68/1000
5710/5710 [==============================] - 0s 13us/step - loss: 3.7144e-05 - val_loss: 3.9531e-05
Epoch 69/1000
5710/5710 [==============================] - 0s 11us/step - loss: 3.6962e-05 - val_loss: 3.8479e-05
Epoch 70/1000
5710/5710 [==============================] - 0s 12us/step - loss: 3.6638e-05 - val_loss: 3.8017e-05
Epoch 71/1000
5710/5710 [==============================] - 0s 13us/step - loss: 3.6216e-05 - val_loss: 3.8852e-05
Epoch 72/1000
5710/5710 [==============================] - 0s 12us/step - loss: 3.6190e-05 - val_loss: 3.7556e-05
Epoch 73/1000
5710/5710 [==============================] - 0s 9us/step - loss: 3.5755e-05 - val_loss: 3.7283e-05
Epoch 74/1000
5710/5710 [==============================] - 0s 14us/step - loss: 3.5150e-05 - val_loss: 3.7096e-05
Epoch 75/1000
5710/5710 [==============================] - 0s 11us/step - loss: 3.4991e-05 - val_loss: 

Epoch 139/1000
5710/5710 [==============================] - 0s 10us/step - loss: 2.4261e-05 - val_loss: 2.3701e-05
Epoch 140/1000
5710/5710 [==============================] - 0s 8us/step - loss: 2.4300e-05 - val_loss: 2.3774e-05
Epoch 141/1000
5710/5710 [==============================] - 0s 11us/step - loss: 2.4422e-05 - val_loss: 2.3602e-05
Epoch 142/1000
5710/5710 [==============================] - 0s 10us/step - loss: 2.4326e-05 - val_loss: 2.3895e-05
Epoch 143/1000
5710/5710 [==============================] - 0s 8us/step - loss: 2.3925e-05 - val_loss: 2.3472e-05
Epoch 144/1000
5710/5710 [==============================] - 0s 9us/step - loss: 2.3820e-05 - val_loss: 2.2931e-05
Epoch 145/1000
5710/5710 [==============================] - 0s 12us/step - loss: 2.3922e-05 - val_loss: 2.2983e-05
Epoch 146/1000
5710/5710 [==============================] - 0s 8us/step - loss: 2.3878e-05 - val_loss: 2.3387e-05
Epoch 147/1000
5710/5710 [==============================] - 0s 12us/step - loss: 2.4

In [17]:
def buildManyToOneModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
  # output shape: (1, 1)
  model.add(Dense(1))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [18]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 30, 1)
X_train, Y_train = shuffle(X_train, Y_train)
# because no return sequence, Y_train and Y_val shape must be 2 dimension
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

model = buildManyToOneModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(30, 10))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5684 samples, validate on 631 samples
Epoch 1/1000
5684/5684 [==============================] - 1s 214us/step - loss: 0.1260 - val_loss: 0.0489
Epoch 2/1000
5684/5684 [==============================] - 1s 95us/step - loss: 0.0198 - val_loss: 0.0065
Epoch 3/1000
5684/5684 [==============================] - 1s 97us/step - loss: 0.0030 - val_loss: 0.0015
Epoch 4/1000
5684/5684 [==============================] - 1s 91us/step - loss: 0.0011 - val_loss: 8.5677e-04
Epoch 5/1000
5684/5684 [==============================] - 1s 94us/st

5684/5684 [==============================] - 1s 93us/step - loss: 4.0844e-05 - val_loss: 4.4955e-05
Epoch 68/1000
5684/5684 [==============================] - 1s 98us/step - loss: 4.1663e-05 - val_loss: 4.5534e-05
Epoch 69/1000
5684/5684 [==============================] - 1s 101us/step - loss: 4.0127e-05 - val_loss: 4.4829e-05
Epoch 70/1000
5684/5684 [==============================] - 1s 96us/step - loss: 4.0075e-05 - val_loss: 4.3987e-05
Epoch 71/1000
5684/5684 [==============================] - 1s 102us/step - loss: 3.9069e-05 - val_loss: 4.3706e-05
Epoch 72/1000
5684/5684 [==============================] - 1s 98us/step - loss: 3.9259e-05 - val_loss: 4.5221e-05
Epoch 73/1000
5684/5684 [==============================] - 1s 95us/step - loss: 3.9645e-05 - val_loss: 4.3180e-05
Epoch 74/1000
5684/5684 [==============================] - 1s 93us/step - loss: 3.9591e-05 - val_loss: 4.4760e-05
Epoch 75/1000
5684/5684 [==============================] - 1s 96us/step - loss: 3.8798e-05 - val_los

5684/5684 [==============================] - 1s 101us/step - loss: 2.9560e-05 - val_loss: 3.2004e-05
Epoch 139/1000
5684/5684 [==============================] - 1s 99us/step - loss: 2.8903e-05 - val_loss: 3.6294e-05
Epoch 140/1000
5684/5684 [==============================] - 1s 93us/step - loss: 2.8462e-05 - val_loss: 3.4719e-05
Epoch 141/1000
5684/5684 [==============================] - 1s 88us/step - loss: 2.8346e-05 - val_loss: 3.0932e-05
Epoch 142/1000
5684/5684 [==============================] - 0s 87us/step - loss: 2.9023e-05 - val_loss: 3.2613e-05
Epoch 143/1000
5684/5684 [==============================] - 1s 91us/step - loss: 2.8558e-05 - val_loss: 3.0731e-05
Epoch 144/1000
5684/5684 [==============================] - 1s 100us/step - loss: 2.8508e-05 - val_loss: 3.4878e-05
Epoch 145/1000
5684/5684 [==============================] - 1s 94us/step - loss: 2.8543e-05 - val_loss: 3.0739e-05
Epoch 146/1000
5684/5684 [==============================] - 1s 93us/step - loss: 2.8331e-05 -

In [19]:
def buildOneToManyModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2]))
  # output shape: (5, 1)
  model.add(Dense(1))
  model.add(RepeatVector(5))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [20]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 1, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

model = buildOneToManyModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, input_shape=(1, 10))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10)                840       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 1)              0         
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5706 samples, validate on 634 samples
Epoch 1/1000
5706/5706 [==============================] - 1s 155us/step - loss: 0.0716 - val_loss: 0.0550
Epoch 2/1000
5706/5706 [==============================] - 0s 12us/step - loss: 0.0420 - val_loss: 0.0293
Epoch 3/1000
5706/5706 [==============================] - 0s 11us/step - loss: 0.0200 - val_loss: 0.0113
Epoch 4/1000
5706/5706 [=====================

5706/5706 [==============================] - 0s 10us/step - loss: 7.2693e-05 - val_loss: 7.1584e-05
Epoch 67/1000
5706/5706 [==============================] - 0s 10us/step - loss: 7.2095e-05 - val_loss: 7.2374e-05
Epoch 68/1000
5706/5706 [==============================] - 0s 10us/step - loss: 7.1902e-05 - val_loss: 7.0978e-05
Epoch 69/1000
5706/5706 [==============================] - 0s 8us/step - loss: 7.1651e-05 - val_loss: 7.0416e-05
Epoch 70/1000
5706/5706 [==============================] - 0s 12us/step - loss: 7.1334e-05 - val_loss: 6.9437e-05
Epoch 71/1000
5706/5706 [==============================] - 0s 10us/step - loss: 7.0922e-05 - val_loss: 6.9515e-05
Epoch 72/1000
5706/5706 [==============================] - 0s 9us/step - loss: 7.0289e-05 - val_loss: 7.0556e-05
Epoch 73/1000
5706/5706 [==============================] - 0s 10us/step - loss: 7.0702e-05 - val_loss: 6.9047e-05
Epoch 74/1000
5706/5706 [==============================] - 0s 9us/step - loss: 7.0314e-05 - val_loss: 6.

5706/5706 [==============================] - 0s 12us/step - loss: 5.7315e-05 - val_loss: 5.5001e-05
Epoch 138/1000
5706/5706 [==============================] - 0s 11us/step - loss: 5.7564e-05 - val_loss: 5.7054e-05
Epoch 139/1000
5706/5706 [==============================] - 0s 11us/step - loss: 5.7828e-05 - val_loss: 5.5401e-05
Epoch 140/1000
5706/5706 [==============================] - 0s 12us/step - loss: 5.7567e-05 - val_loss: 5.5924e-05
Epoch 141/1000
5706/5706 [==============================] - 0s 11us/step - loss: 5.7272e-05 - val_loss: 5.6636e-05
Epoch 142/1000
5706/5706 [==============================] - 0s 11us/step - loss: 5.7578e-05 - val_loss: 5.5470e-05
Epoch 143/1000
5706/5706 [==============================] - 0s 11us/step - loss: 5.7082e-05 - val_loss: 5.6563e-05
Epoch 144/1000
5706/5706 [==============================] - 0s 13us/step - loss: 5.7268e-05 - val_loss: 5.5443e-05
Epoch 145/1000
5706/5706 [==============================] - 0s 13us/step - loss: 5.7256e-05 - v

In [21]:
def buildManyToManyModel(shape):
  model = Sequential()
  model.add(LSTM(10, input_length=shape[1], input_dim=shape[2], return_sequences=True))
  # output shape: (5, 1)
  model.add(TimeDistributed(Dense(1)))
  model.compile(loss="mse", optimizer="adam")
  model.summary()
  return model

In [22]:
train = readTrain()
train_Aug = augFeatures(train)
train_norm = normalize(train_Aug)
# change the last day and next day 
X_train, Y_train = buildTrain(train_norm, 5, 5)
X_train, Y_train = shuffle(X_train, Y_train)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.1)

# from 2 dimmension to 3 dimension
Y_train = Y_train[:,:,np.newaxis]
Y_val = Y_val[:,:,np.newaxis]

model = buildManyToManyModel(X_train.shape)
callback = EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")
model.fit(X_train, Y_train, epochs=1000, batch_size=128, validation_data=(X_val, Y_val), callbacks=[callback])


D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(10, return_sequences=True, input_shape=(5, 10))`
  This is separate from the ipykernel package so we can avoid doing imports until


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 5, 10)             840       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 5, 1)              11        
Total params: 851
Trainable params: 851
Non-trainable params: 0
_________________________________________________________________
Train on 5703 samples, validate on 633 samples
Epoch 1/1000
5703/5703 [==============================] - 1s 193us/step - loss: 0.0997 - val_loss: 0.0552
Epoch 2/1000
5703/5703 [==============================] - 0s 26us/step - loss: 0.0360 - val_loss: 0.0169
Epoch 3/1000
5703/5703 [==============================] - 0s 21us/step - loss: 0.0125 - val_loss: 0.0087
Epoch 4/1000
5703/5703 [==============================] - 0s 23us/step - loss: 0.0081 - val_loss: 0.0067
Epoch 5/1000
5703/5703 [==============================] - 0s 27us/step -

5703/5703 [==============================] - 0s 23us/step - loss: 1.0266e-04 - val_loss: 1.0648e-04
Epoch 69/1000
5703/5703 [==============================] - 0s 21us/step - loss: 1.0089e-04 - val_loss: 1.0763e-04
Epoch 70/1000
5703/5703 [==============================] - 0s 23us/step - loss: 1.0065e-04 - val_loss: 1.0638e-04
Epoch 71/1000
5703/5703 [==============================] - 0s 24us/step - loss: 1.0002e-04 - val_loss: 1.0516e-04
Epoch 72/1000
5703/5703 [==============================] - 0s 20us/step - loss: 9.9747e-05 - val_loss: 1.0401e-04
Epoch 73/1000
5703/5703 [==============================] - 0s 20us/step - loss: 9.9285e-05 - val_loss: 1.0538e-04
Epoch 74/1000
5703/5703 [==============================] - 0s 21us/step - loss: 9.8778e-05 - val_loss: 1.0456e-04
Epoch 75/1000
5703/5703 [==============================] - 0s 20us/step - loss: 9.8190e-05 - val_loss: 1.0309e-04
Epoch 76/1000
5703/5703 [==============================] - 0s 21us/step - loss: 9.7798e-05 - val_loss:

5703/5703 [==============================] - 0s 21us/step - loss: 8.8541e-05 - val_loss: 9.5509e-05
Epoch 140/1000
5703/5703 [==============================] - 0s 21us/step - loss: 8.8916e-05 - val_loss: 9.7078e-05
Epoch 141/1000
5703/5703 [==============================] - 0s 18us/step - loss: 8.7924e-05 - val_loss: 9.5896e-05
Epoch 142/1000
5703/5703 [==============================] - 0s 23us/step - loss: 8.8650e-05 - val_loss: 9.9337e-05
Epoch 143/1000
5703/5703 [==============================] - 0s 21us/step - loss: 8.7882e-05 - val_loss: 9.6416e-05
Epoch 144/1000
5703/5703 [==============================] - 0s 21us/step - loss: 8.9488e-05 - val_loss: 9.8844e-05
Epoch 145/1000
5703/5703 [==============================] - 0s 21us/step - loss: 8.8746e-05 - val_loss: 9.5390e-05
Epoch 146/1000
5703/5703 [==============================] - 0s 20us/step - loss: 8.8480e-05 - val_loss: 9.5477e-05
Epoch 147/1000
5703/5703 [==============================] - 0s 19us/step - loss: 8.9817e-05 - v